## 整理数据集

把数据集放到 datases/raw/ 目录下，建两个文件夹 y 和 n

- datasets/raw/y/ 里面放技能 ready 的图
- datasets/raw/n/ 里面放没技能的图

要求输入图片尺寸为 3x64x64，文件名随便

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

def default_loader(path):
    return Image.open(path).convert('RGB')

class SkillDataset(Dataset):
    def __init__(self, path: Path) -> None:
        super().__init__()
        self.y = list((path / 'y').glob('**/*'))
        self.n = list((path / 'n').glob('**/*'))
        self.transform = transforms.ToTensor()
        self.loader = default_loader
        # 技能图片没多大，一次性全部载入内存算了
        self.data = [ self.get(i) for i in range(len(self))]
    
    def __len__(self):
        return len(self.y) + len(self.n)
    
    def get(self, index):
        if index < len(self.y):
            if index % 100 == 0:
                print(f'load y: {index} / {len(self.y)}')
            path = self.y[index]
            label = 1
        else:
            if index % 100 == 0:
                print(f'load n: {index - len(self.y)} / {len(self.n)}')
            path = self.n[index - len(self.y)]
            label = 0
        image = self.loader(path)
        image = self.transform(image)
        return image, label
    
    def __getitem__(self, index):
        return self.data[index]

raw_path = Path("datasets/raw")
dataset = SkillDataset(raw_path)

In [ ]:

import torch

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 显存不够可以把 batch size 改小点
train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False, num_workers=0)

In [ ]:
import torch

class InceptionA(torch.nn.Module):
    def __init__(self, in_ch) -> None:
        super().__init__()
        self.branch_1x1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)

        self.branch_5x5_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch_3x3_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch_3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = torch.nn.Conv2d(in_ch, 24, kernel_size=1)

    def forward(self, x):
        branch_1x1 = self.branch_1x1(x)

        branch_5x5 = self.branch_5x5_1(x)
        branch_5x5 = self.branch_5x5_2(branch_5x5)

        branch_3x3 = self.branch_3x3_1(x)
        branch_3x3 = self.branch_3x3_2(branch_3x3)
        branch_3x3 = self.branch_3x3_3(branch_3x3)

        branch_pool = torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch_1x1, branch_5x5, branch_3x3, branch_pool] # 16 + 24 + 24 + 24
        return torch.cat(outputs, 1)


class GoogleNet(torch.nn.Module):
    def __init__(self, channels) -> None:
        super().__init__()
        self.conv1 = torch.nn.Conv2d(channels, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        self.incep1 = InceptionA(10)
        self.incep2 = InceptionA(20)
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(14872, 2)

    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = torch.nn.functional.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


In [ ]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if not use_cuda:
    print("WARNING: CPU will be used for training.")

model = GoogleNet(3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

start_time = time.time()
def train(epoch):
    global start_time
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
    cur_time = time.time()
    cost = cur_time - start_time
    print(f'Train Epoch: {epoch}, Loss: {loss.item():.8f}, cost: {cost:.2f} s')
    start_time = cur_time
            
def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)

    print(f'=== Test: Loss: {test_loss:.8f}, Acc: {acc:.4f} ===')
    return test_loss, acc

In [ ]:
if use_cuda:
    torch.cuda.empty_cache()

output = Path('checkpoints')
output.mkdir(exist_ok=True, parents=True)

best_epoch = 0
best_loss = 100.0
best_acc = 0.0
default_interval = 10

def pipeline(start_epoch = 0, test_interval = default_interval):
    global best_epoch, best_loss, best_acc

    for epoch in range(start_epoch, 1000):
        train(epoch)
        if epoch % test_interval != 0:
            continue
        
        loss, acc = test()
        print(f'=== Pre best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ===')
        torch.save(model, output / f'model_{epoch}.pt')
        if loss > best_loss:
            if epoch - best_epoch > test_interval * 10:
                print('No improvement for a long time, Early stop!')
                break
            else:
                continue
        best_epoch = epoch
        best_loss = loss
        best_acc = acc
        print(f'====== New best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ======')
        torch.save(model, output / 'best.pt')

pipeline()

In [ ]:
model = torch.load(output / f'best.pt')
test()

In [ ]:
# 导出 onnx

import torch.onnx
from pathlib import Path


def convert_onnx(path: Path):
    model = torch.load(path, map_location=torch.device("cpu"))
    model.eval()
    dummy_input = torch.randn(1, 3, 64, 64)
    torch.onnx.export(
        model,
        dummy_input,
        path.with_suffix(".onnx"),
        input_names=["input"],
        output_names=["output"],
    )


convert_onnx(output / "best.pt")


In [ ]:
import onnx

onnx.checker.check_model(str(output / "best.onnx"))

## 以下是部分清洗数据的代码，可以自己研究下

In [ ]:
# 随机数据挑选，请手工清洗

# from pathlib import Path
# import random

# clean_set_size = 1000
# raw_path = Path("datasets/raw")
# positive_set = random.sample(list((raw_path / "y").glob("**/*")), clean_set_size)
# negative_set = random.sample(list((raw_path / "n").glob("**/*")), clean_set_size)

# clean = Path("datasets/clean/")
# clean_y = clean / "y"
# clean_y.mkdir(parents=True, exist_ok=True)
# clean_n = clean / "n"
# clean_n.mkdir(parents=True, exist_ok=True)
# for path in positive_set:
#     path.rename(clean_y / path.name)
# for path in negative_set:
#     path.rename(clean_n / path.name)


In [ ]:
# class SkillRawDataset(Dataset):
#     def __init__(self) -> None:
#         super().__init__()
#         self.y = list(Path('datasets/raw/y/').glob('**/*'))
#         self.n = list(Path('datasets/raw/n/').glob('**/*'))
#         self.transform = transforms.ToTensor()
#         self.loader = default_loader
#         self.data = [ self.get(i) for i in range(len(self))]
    
#     def __len__(self):
#         return len(self.y) + len(self.n)
    
#     def get(self, index):
#         # print(f'load: {self.count} / {len(self)}')
#         if index < len(self.y):
#             path = self.y[index]
#             label = 1
#         else:
#             path = self.n[index - len(self.y)]
#             label = 0
#         image = self.loader(path)
#         image = self.transform(image)
#         return image, label
    
#     def __getitem__(self, index):
#         return self.data[index]
    
#     def get_path(self, index):
#         if index < len(self.y):
#             path = self.y[index]
#             label = 1
#         else:
#             path = self.n[index - len(self.y)]
#             label = 0
#         return path, label

# raw_data_set = SkillRawDataset()


In [ ]:

# raw_loader = DataLoader(raw_data_set, batch_size=1, shuffle=False, num_workers=0)
# import os
# import shutil
# def clear():
#     model.eval()
#     test_loss = 0
#     Path('datasets/maybe_error/1').mkdir(parents=True, exist_ok=True)
#     Path('datasets/maybe_error/0').mkdir(parents=True, exist_ok=True)
#     with torch.no_grad():
#         for batch_idx, (data, target) in enumerate(raw_loader):
#             data, target = data.cuda(), target.cuda()
#             output = model(data)
#             loss = criterion(output, target).item() # sum up batch loss
#             test_loss += loss
#             pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
#             correct = pred.eq(target.view_as(pred)).sum().item()
#             if not correct:
#                 tup = raw_data_set.get_path(batch_idx)
#                 print(tup)
#                 os.rename(tup[0], Path('datasets/maybe_error/') / str(tup[1]) / tup[0].name)                
                


# print(len(raw_data_set))
# clear()